# Image testing for pre-processing

## Importing Modules

In [1]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import csv
import json
from os import listdir
from os.path import isfile, join

from ManageLeafData import ManageLeafData as mld

## Loading image

In [2]:
img_1 = cv.imread('IMG_20230922_171310509.jpg')
cv.imshow('Raw Image', img_1)
cv.waitKey(0)
cv.destroyAllWindows()

In [3]:
image_lst = ['IMG_20230922_171255218.jpg', 'IMG20230922165200.jpg', 'IMG20230922170108.jpg', 'IMG20230922170842.jpg', 'IMG20231004155721.jpg']

## Image Smoothening

In [4]:
def GaussianSmoothing(mat):
    _pi = 3.14
    _sd = np.std(mat)
    _e = 2.718
    for y in range(len(mat)):
        for x in range(len(mat[y])):
            try:
                mat[y][x] = ((1 / (2 * _pi * (_sd ** 2)))) * (_e ** (((x ** 2) + (y ** 2)) * (1 / (2 * (x ** 2)))))
            except ZeroDivisionError:
                pass
    return mat

In [5]:
# img_1_myGblur = GaussianSmoothing(cv.cvtColor(img_1, cv.COLOR_BGR2GRAY))

# img_1_myGblur_resized = cv.resize(img_1_myGblur, (1000, 1000))
# cv.imshow('Image Gaussian Blur & Resized', img_1_myGblur_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [6]:
# img_1_resized_Gblur = cv.GaussianBlur(img_1_resized, (7,7), 0)
# cv.imshow('Image Resized & Gaussian Blur', img_1_resized_Gblur)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur = cv.GaussianBlur(img_1, (7,7), 0)
cv.imshow('Image Gaussian Blur', img_1_Gblur)
cv.waitKey(0)
cv.destroyAllWindows()

## Image Resizing

In [7]:
# img_1_resized = cv.resize(img_1, (int(img_1.shape[1] / 6), int(img_1.shape[0] / 6)))
# cv.imshow('Image Resized', img_1_resized)
# cv.waitKey(0)
# cv.destroyAllWindows()

img_1_Gblur_resized = cv.resize(img_1_Gblur, (1000, 1000))
cv.imshow('Image Gaussian Blur & Resized', img_1_Gblur_resized)
cv.waitKey(0)
cv.destroyAllWindows()

In [8]:
img_1_Gblur_resized_gray = cv.cvtColor(img_1_Gblur_resized, cv.COLOR_BGR2GRAY)
cv.imshow('Image Gaussian Blur & Resized & Gray', img_1_Gblur_resized_gray)
cv.waitKey(0)
cv.destroyAllWindows()

In [9]:
# Function to map each intensity level to output intensity level. 
def pixelVal(pix, r1, s1, r2, s2): 
    if (0 <= pix and pix <= r1): 
        return (s1 / r1)*pix 
    elif (r1 < pix and pix <= r2): 
        return ((s2 - s1)/(r2 - r1)) * (pix - r1) + s1 
    else: 
        return ((255 - s2)/(255 - r2)) * (pix - r2) + s2 

# Define parameters. 
r1 = 110 # controls darkness
s1 = 0.15 # lower bound
r2 = 7 # controls lightness
s2 = 250 # upper bound

# Vectorize the function to apply it to each value in the Numpy array. 
pixelVal_vec = np.vectorize(pixelVal) 

# Apply contrast stretching. 
img_1_contrast_stretched = pixelVal_vec(img_1_Gblur_resized_gray, r1, s1, r2, s2) 

cv.imshow('Contrast Stretching', img_1_contrast_stretched)
cv.waitKey(0)
cv.destroyAllWindows()


In [10]:
img_thresh = cv.threshold(img_1_Gblur_resized_gray, 127, 255, cv.THRESH_BINARY)
cv.imshow('Thresholding', img_thresh[1])
cv.waitKey(0)
cv.destroyAllWindows()

# print(img_thresh)

In [11]:
img_thresh1 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 0)
cv.imshow('Thresholding', img_thresh1)
# cv.waitKey(0)
# cv.destroyAllWindows()

In [12]:
img_thresh1 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 3)
cv.imshow('Thresholding1', img_thresh1)
img_thresh2 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 4)
cv.imshow('Thresholding2', img_thresh2)
img_thresh3 = cv.adaptiveThreshold(img_1_Gblur_resized_gray, 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 5)
cv.imshow('Thresholding3', img_thresh3)
cv.waitKey(0)
cv.destroyAllWindows()

In [13]:
img_hist = cv.equalizeHist(img_thresh)
cv.imshow('Histogram equlization', img_hist)
cv.waitKey(0)
cv.destroyAllWindows()

error: OpenCV(4.7.0) :-1: error: (-5:Bad argument) in function 'equalizeHist'
> Overload resolution failed:
>  - src is not a numerical tuple
>  - Expected Ptr<cv::UMat> for argument 'src'


## Combining these processes in a single function

In [14]:
def ImagePreprocess_Combination_1(name: str):
    cv.imshow('Final Image', pixelVal_vec(cv.resize(cv.GaussianBlur(cv.imread(name)
                                           , (7,7), 0), 
                           (1000, 1000)), 
                r1, s1, r2, s2))
    cv.waitKey(0)
    cv.destroyAllWindows()

In [30]:
def ImagePreprocess_Combination_2(name: str):
    cv.imshow('Final Image', cv.equalizeHist(cv.adaptiveThreshold(cv.cvtColor(cv.resize(cv.GaussianBlur(cv.imread(name)
                                                                                                        , (7,7), 0)
                                                                                        , (1000, 1000))
                                                                              , cv.COLOR_BGR2GRAY)
                                                                  , 255, cv.ADAPTIVE_THRESH_MEAN_C, cv.THRESH_BINARY, 199, 5)
                                            )
             )
    cv.waitKey(0)
    cv.destroyAllWindows()

In [25]:
for image in image_lst:
    ImagePreprocess_Combination_2(image)

In [26]:
import pandas as pd

df = pd.read_csv('Leaf_Data_Mapped.csv')
df

,Image,Common name,Scientific name,Class number
0,IMG20231004154634.jpg,Peepla,Ficus religiosa,0
1,IMG20231004154651.jpg,Peepla,Ficus religiosa,0
2,IMG20231004154748.jpg,Peepla,Ficus religiosa,0
3,IMG20231004154809.jpg,Peepla,Ficus religiosa,0
4,IMG20231004154917.jpg,Peepla,Ficus religiosa,0
...,...,...,...,...
92,IMG20231004172618.jpg,Pipal,Ficus amplissima,19
93,IMG20231004172657.jpg,Pipal,Ficus amplissima,19
94,IMG20231004172723.jpg,Pipal,Ficus amplissima,19
95,IMG20231004172803.jpg,Pipal,Ficus amplissima,19


In [27]:
for image in df['Image']:
    ImagePreprocess_Combination_2('Natural/' + image)

In [33]:
image_blackBG_lst = mld.getImageNameList('BlackBG/')
image_blackBG_lst

['IMG_20231004_160720.jpg',
 'IMG_20231004_161958.jpg',
 'IMG_20231004_162249.jpg',
 'IMG_20231004_162417.jpg',
 'IMG_20231004_162740.jpg',
 'IMG_20231004_163320.jpg',
 'IMG_20231004_164307.jpg',
 'IMG_20231004_165618.jpg',
 'IMG_20231004_170900.jpg',
 'IMG_20231004_171728.jpg']

In [34]:
for image in image_blackBG_lst:
    ImagePreprocess_Combination_2('BlackBG/' + image)

## Getting name of the images in a list.

In [ ]:
myPath = "Collected Leaf data/Botany Collection/Natural/"

## Mapping images of 'Botany Collection/Natural/'

## Mapping completed and stored in Lead_Data_Mapped.csv file

## Testing ManageLeafData

In [ ]:
# mld.createSpeciesJsonFile()

In [ ]:
# mld.readSpeciesJsonFile()